In [1]:
import numpy as np
import pandas as pd
import urllib.request, urllib.parse,urllib.error
from bs4 import BeautifulSoup
import time
from requests_html import HTMLSession, AsyncHTMLSession
import ssl
import re
import random
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
# data frame consists of first 100 pages of https://boardgamegeek.com/browse/boardgame
# 100 pages where each page has 100 games = 10 000 games to work with
# it is for beginning, we can get the rest easily  :) 

games_list = pd.read_csv(r'C:/Users/Armit/Python/Scripts/Project/games_100_pages.csv', sep = '\t', encoding='utf-8')

In [6]:
games_list = games_list[games_list['categories'].isna()]

In [ ]:
games_list = games_list.drop('Unnamed: 0', axis=1)

In [4]:
games_list['overview_url'] = 'https://boardgamegeek.com' + games_list['url']

In [5]:
games_list['overview_url'][1591]

'https://boardgamegeek.com/boardgame/111105/agents-smersh'

In [5]:
games_list['categories'] = pd.Series().astype(np.uint8)

<ipython-input-5-cabcd76cb407>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  games_list['categories'] = pd.Series().astype(np.uint8)


In [15]:
games_list[2152:2695].to_csv(r'./games_list_categories2152_2695_.csv', sep='\t', encoding='utf-8', header='true',index=False)

In [24]:
games_list[games_list['categories'].isna() == False].to_csv(r'./games_list_categories_additional.csv', sep='\t', encoding='utf-8', header='true',index=False)

In [14]:
games_list.reset_index(inplace=True)

In [15]:
#2152,3125,1
for i in range(len(games_list)):
    url = games_list['overview_url'][i]
    browser = webdriver.Firefox()
    # opening url in browser via selenium
    browser.get(url)
    # checking if we have beed redirected
    soup = BeautifulSoup(browser.page_source)
    browser.quit()
    #soup = BeautifulSoup(resp, "lxml")
    tags = soup('a')
    category = []
    for tag in tags:
        if tag.get('class') == ['ng-binding'] and 'boardgamecategory/' in str(tag.get('href')):
            #print('Contents:', tag.contents[0])
            category.append(tag.contents[0])
            games_list['categories'][i] = category

<ipython-input-15-641310459ed5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_list['categories'][i] = category


### Test and experiments below

In [6]:
url = 'https://boardgamegeek.com/boardgame/174430/gloomhaven'
browser = webdriver.Firefox()
# opening url in browser via selenium
browser.get(url)
# checking if we have beed redirected
soup = BeautifulSoup(browser.page_source)
browser.quit()
#soup = BeautifulSoup(resp, "lxml")
tags = soup('a')

for tag in tags:
    if tag.get('class') == ['ng-binding'] and 'boardgamecategory/' in str(tag.get('href')):
       #print('Contents:', tag.contents[0])
        print(tag.contents[0])
     

Adventure
Exploration
Fantasy
Fighting
Miniatures


In [ ]:
for i in range(136,len(games_list),1):
    url = games_list['overview_url'][i]
    asession = AsyncHTMLSession()
    r = await asession.get(url)
    await r.html.arender(timeout = 8)
    resp=r.html.raw_html
    soup = BeautifulSoup(resp, "lxml")
    tags = soup('span')
    category = []
    for tag in tags:
        if tag.get('class') == ['ng-binding'] and 'boardgamecategory/' in str(tag.get('href')):
           #print('Contents:', tag.contents[0])
            category.append(tag.contents[0])
            games_list['categories'][i] = category

<ipython-input-5-7b4a7e12eda8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_list['categories'][i] = category


In [11]:
tags = soup('span')
category = []
for tag in tags:
    if tag.get('class') == ['ng-isolate-scope']: # and 'boardgamecategory/' in str(tag.get('href')):
    #if 'boardgamecategory' in tag:
        if 'playerage' in str(tag):
            print('TAG:', tag)
            print('TEXT:', str(tag.get_text()).strip().split(':')[1])
        

TAG: <span class="ng-isolate-scope" item-poll-button="playerage" poll-view="results"> <button class="btn btn-link btn-xs" tooltip-append-to-body="true" tooltip-popup-delay="500" uib-tooltip="View poll and results"> 							Community: <span class="ng-binding" ng-bind-html="geekitemctrl.geekitem.data.item.polls.playerage|to_trusted">14+</span> </button> </span>
TEXT:  14+


In [66]:
url = 'https://boardgamegeek.com/boardgame/174430/gloomhaven'

In [67]:
asession = AsyncHTMLSession()
r = await asession.get("https://boardgamegeek.com/boardgame/174430/gloomhaven")
await r.html.arender()
resp=r.html.raw_html

In [69]:
tags = soup('a')
category = []
for tag in tags:
    if tag.get('class') == ['ng-binding'] and 'boardgamecategory/' in str(tag.get('href')):
    #if 'boardgamecategory' in tag:
        #print('TAG:', tag)
        #game_name.append(tag.next_element)
        #game_id.append(tag.get('href', None).split('/')[2])
        #game_url.append(tag.get('href', None))
               
    #print('next element:', tag.next_element)
        #print('URL:', tag.get('href', None))
        #print('Game_id:', tag.get('href', None).split('/')[2])
        
        print('Contents:', tag.contents[0])
        category.append(tag.contents[0])
        #print('Attrs:', tag.attrs)
        #print('Class:', tag.get('class'))
        #print('href',tag.get('href', None))
    #print('Class:', tag.get('title'))

Contents: Adventure
Contents: Exploration
Contents: Fantasy
Contents: Fighting
Contents: Miniatures
